实战需求

利用人工神经网络算法，预测中国石化的涨跌行情，然后再将预测结果与实际情况进行对比，判断预测对错。

In [5]:
import talib   #导入talib库
from jqdata import *  #导入聚宽函数库
from pybrain.datasets import ClassificationDataSet  #从pybrain的数据集中导入分类数据集

In [4]:
test_stock = '600028.XSHG'  #设置测试标的为中国石化
start_date = datetime.date(2023, 1, 4)  #设置开始时间
end_date = datetime.date(2023, 4, 21)   #设置结束时间

In [6]:
trading_days = list(get_all_trade_days())  #获取所有交易日
start_date_index = trading_days.index(start_date)  #获取开始时间
end_date_index = trading_days.index(end_date)   #获取结束时间

In [7]:
#训练数据
#这是一个分类数据集，参数有3个,分别是输入的特征的数目、输出的结果的数目、类别的数目
#输入的特征的数目有2个，输出的结果只有一个结果，其中的类别是涨或者跌，所以参数是2，1，2
train_data = ClassificationDataSet(2, 1, nb_classes=2)
# 测试数据
test_data = ClassificationDataSet(2, 1, nb_classes=2)

In [10]:
for index in range(start_date_index, end_date_index):
    # 得到计算指标的所有数据
    start_day = trading_days[index - 30]
    end_day = trading_days[index]
    
    #利用get_price()函数获得股票数据
    stock_data = get_price(test_stock,start_day,end_day,
                           frequency='daily', fields=['close'])
    
    #变义变量并赋值为收盘价
    close_prices = stock_data['close'].values
    
    #通过数据计算指标
    # -2是保证获取的数据是昨天的，-1就是通过今天的数据计算出来的指标
    sma = talib.SMA(close_prices)[-2]  #简单移动平均线指标
    wma = talib.WMA(close_prices)[-2]  #加权移动平均数指标
    
    #训练数据或者测试数据的输入特征
    features = []
    features.append(sma)
    features.append(wma)
    
    # 训练数据或者测试的标签数据，就是涨或者跌，涨用1表示，平或者跌用0表示
    label = 0
    if close_prices[-1] > close_prices[-2]:
       label = 1
    
    # 添加训练数据和测试数据
    if index < end_date_index - 1:
        train_data.addSample(features, [label])
    else:
        test_data.addSample(features, [label])

    #采用独热编码将离散标称值转换为数值变量
    train_data._convertToOneOfMany( )
    test_data._convertToOneOfMany( )      

In [1]:
#导入需要的数据包
from pybrain.tools.shortcuts import buildNetwork
from pybrain.supervised.trainers import BackpropTrainer
from pybrain.structure.modules  import SoftmaxLayer
from pybrain.utilities      import percentError

ModuleNotFoundError: No module named 'structure'

In [12]:
#创建神经网络
fnn = buildNetwork( train_data.indim, 5, train_data.outdim, outclass=SoftmaxLayer)

#训练神经网络
trainer = BackpropTrainer( fnn, dataset=train_data, momentum=0.1, verbose=True, weightdecay=0.01)

#训练一次，可以通过括号里面的数字调节训练的次数
trainer.trainEpochs( 1 )

# 计算预测误差，这个是误差，不是预测准确率，0表示预测对了，100%表示预测错了
tstresult = percentError( trainer.testOnClassData(dataset=test_data ), 
                         test_data['class'] )


print("预测误差:",tstresult)
if tstresult==0 :
    print("预测正确！")
else :
    print("预测错误！")

Total error:  0.138692077971
预测误差: 0.0
预测正确！
